In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as mpl
import scipy.spatial.distance as ssd
import scipy.cluster.hierarchy as sch
import random
import statsmodels.api as sm
import math

本脚本为对股指成份股的回测脚本。窗口长度和调仓频率可在goBacktest()中调整。

In [2]:
def filter_extreme_MAD(df): 
    #MAD: 中位数去极值   
    savemore = 4.7 #控制截断程度的参数
    saveless = 3.9
    for fac_series in df.columns:
        LocSkew = df[fac_series].skew()
        median = df[fac_series].median()
        new_median = ( (df[fac_series] - median).abs() ).median()
        #偏度适应得找上下界限:正偏则对右边heavytail更容忍一些
        if LocSkew >= 0:
            UpLimit = median + savemore * new_median
            LowLimit = median - saveless * new_median
        else:
            UpLimit = median + saveless * new_median
            LowLimit = median - savemore * new_median
        #截取当前fac_series
        df[fac_series] = df[fac_series].clip(LowLimit , UpLimit)

    return df

In [3]:
def filter_extreme_perc( array, min = 0, max = 100): #百分位法去极值
    a = array.copy()
    lower = np.percentile(a,min)
    upper = np.percentile(a,max)
    l_mask = a < lower
    u_mask = a > upper
    a[l_mask] = lower
    a[u_mask] = upper
    
    return a

In [45]:
#读取文件
df_facExpose = pd.read_hdf('data.facExpose_1011', 'data')  #读取文件
df_pct = pd.read_hdf('data.pct_1029_zz', 'data')  #读取文件,成份股

In [46]:
df_facExpose

,tradedate,stockcode,Size,Valuation,Earning,Growth,Momentum,Reverse,Volatility,Liquidity,Beta
0,20160901,000001,2.68827,0.17447,-0.17825,0.03010,-0.29377,-0.09287,-0.54104,0.37894,0.29252
1,20160901,000002,2.71749,-0.51404,0.73193,-0.11733,0.42814,0.78411,3.54368,2.73161,-0.74584
2,20160901,000004,-0.99217,-2.11066,-0.83232,0.66871,0.50814,-0.43491,1.66371,-0.15279,0.83037
3,20160901,000005,-0.03793,-0.98772,0.46006,2.21151,0.17442,0.03758,-0.80051,-0.05719,-0.03910
4,20160901,000006,0.55227,0.34243,-0.00022,-1.41225,-0.34026,-1.16306,-0.90739,0.74893,0.32077
...,...,...,...,...,...,...,...,...,...,...,...
1461365,20180830,603993,2.65273,-0.51771,0.13364,1.16279,-0.00696,-1.28440,-0.00297,1.13171,2.53948
1461366,20180830,603996,-1.35375,1.07502,0.24460,0.97376,-1.38243,-0.96278,0.11154,0.69945,-1.58012
1461367,20180830,603997,0.50909,-1.31944,1.64435,-0.01287,0.56967,0.26027,-1.12715,0.43135,-1.29654
1461368,20180830,603998,-0.37490,0.33028,-0.58874,0.23608,-1.82696,-0.33702,-0.34703,-1.02541,0.57967


columns: date, stockcode, various security-level style factor exposures

In [47]:
df_pct

,tradedate,stockcode,pct
0,20160901,000006,-3.7647
1,20160901,000008,-0.9524
2,20160901,000012,-2.7984
3,20160901,000021,-1.5469
4,20160901,000028,0.0000
...,...,...,...
243495,20180830,603877,-0.6911
243496,20180830,603883,0.3664
243497,20180830,603885,-1.9461
243498,20180830,603888,-2.3327


columns: date, stockcode, dailyreturn (unit: %)

In [48]:
def offsetPCT( dftoday, pre_insert_date = '20160831'):
    dftoday = dftoday.set_index(['tradedate','stockcode']) # multi-level index: tradedate, stockcode
    todayIndex = list( dftoday.index.levels[0] )
    ydayIndex = list( dftoday.index.levels[0][:-1].insert(0, pre_insert_date) )
    #绘制位移前后indexlist的映射关系
    mapping = dict( zip(todayIndex,ydayIndex) )

    #完成映射
    df_off = dftoday.rename(index=mapping)
    
    return df_off

In [49]:
offsetPCT(df_pct)

pct
tradedate stockcode        
20160831  000006    -3.7647
          000008    -0.9524
          000012    -2.7984
          000021    -1.5469
          000028     0.0000
...                     ...
20180829  603877    -0.6911
          603883     0.3664
          603885    -1.9461
          603888    -2.3327
          603899    -1.6129

[243500 rows x 1 columns]

In [50]:
df_tomor = offsetPCT(df_pct)       #前一日索引对应次日涨跌幅

In [51]:
df_tomor.rename(columns={'pct':'tmr_pct'},inplace=True)

In [52]:
df_tomor

tmr_pct
tradedate stockcode         
20160831  000006     -3.7647
          000008     -0.9524
          000012     -2.7984
          000021     -1.5469
          000028      0.0000
...                      ...
20180829  603877     -0.6911
          603883      0.3664
          603885     -1.9461
          603888     -2.3327
          603899     -1.6129

[243500 rows x 1 columns]

In [53]:
zzIndex = df_tomor.index.levels[1] #成份股stockcode
zzIndex

Index(['000006', '000008', '000009', '000012', '000021', '000025', '000027',
       '000028', '000030', '000031',
       ...
       '603799', '603806', '603816', '603866', '603868', '603877', '603883',
       '603885', '603888', '603899'],
      dtype='object', name='stockcode', length=685)

In [54]:
df_tomor = df_tomor.reset_index()  #还原为自然索引以便后续合成数据

In [55]:
df_facExpose = df_facExpose.set_index('stockcode')
df_facExpose = df_facExpose.loc[zzIndex] #截取成份股因子值数据表
df_facExpose

,tradedate,Size,Valuation,Earning,Growth,Momentum,Reverse,Volatility,Liquidity,Beta
stockcode,,,,,,,,,,
000006,20160901,0.55227,0.34243,-0.00022,-1.41225,-0.34026,-1.16306,-0.90739,0.74893,0.32077
000006,20160902,0.54421,0.36321,0.00254,-1.41105,-0.19598,-0.85688,-0.95279,0.68112,0.31819
000006,20160905,0.56074,0.32205,-0.00025,-1.41113,-0.35206,-0.13096,-0.91884,0.67310,0.26787
000006,20160906,0.54099,0.34848,0.00160,-1.41073,-0.43923,-0.71668,-0.92382,0.63418,0.24373
000006,20160907,0.53983,0.35007,0.00230,-1.41015,-0.40455,-0.61142,-0.93036,0.60580,0.24564
...,...,...,...,...,...,...,...,...,...,...
603899,20180824,1.81085,-2.12704,1.81578,0.03143,1.53925,2.27664,0.86943,-0.08913,1.72896
603899,20180827,1.80145,-2.10344,1.79570,0.06107,1.45503,1.45226,0.80907,-0.10210,1.54623
603899,20180828,1.80904,-2.11905,1.78633,0.06343,1.46865,1.98108,0.78823,-0.09952,1.54535


In [56]:
df_facExpose.reset_index(inplace=True)
df_facExpose

,stockcode,tradedate,Size,Valuation,Earning,Growth,Momentum,Reverse,Volatility,Liquidity,Beta
0,000006,20160901,0.55227,0.34243,-0.00022,-1.41225,-0.34026,-1.16306,-0.90739,0.74893,0.32077
1,000006,20160902,0.54421,0.36321,0.00254,-1.41105,-0.19598,-0.85688,-0.95279,0.68112,0.31819
2,000006,20160905,0.56074,0.32205,-0.00025,-1.41113,-0.35206,-0.13096,-0.91884,0.67310,0.26787
3,000006,20160906,0.54099,0.34848,0.00160,-1.41073,-0.43923,-0.71668,-0.92382,0.63418,0.24373
4,000006,20160907,0.53983,0.35007,0.00230,-1.41015,-0.40455,-0.61142,-0.93036,0.60580,0.24564
...,...,...,...,...,...,...,...,...,...,...,...
324002,603899,20180824,1.81085,-2.12704,1.81578,0.03143,1.53925,2.27664,0.86943,-0.08913,1.72896
324003,603899,20180827,1.80145,-2.10344,1.79570,0.06107,1.45503,1.45226,0.80907,-0.10210,1.54623
324004,603899,20180828,1.80904,-2.11905,1.78633,0.06343,1.46865,1.98108,0.78823,-0.09952,1.54535
324005,603899,20180829,1.80127,-2.11133,1.77523,0.05007,1.40331,1.36293,0.76723,-0.10967,1.54826


In [57]:
#合并数据表
df_total = pd.merge(df_facExpose, df_pct ,on = ['tradedate', 'stockcode'])
df_total = pd.merge(df_total , df_tomor ,on = ['tradedate', 'stockcode'])
df_total = df_total.set_index(['tradedate','stockcode'])
nfacs = ['Size', 'Valuation', 'Earning', 'Growth', 'Momentum', 'Reverse', 'Volatility', 'Liquidity', 'Beta']
#对每日数据的stock_universe做缺失值填充操作
df_total = df_total.groupby(level = 'tradedate').apply(lambda x: x.fillna(x.median()))
#对每日数据的stock_universe做相应的去极值操作，可用不同方法
df_total[nfacs] = df_total[nfacs].groupby(level = 'tradedate').apply(filter_extreme_MAD)
#对去极值后的数据做标准化
df_total[nfacs] = df_total[nfacs].groupby(level = 'tradedate').apply(lambda x: (x - x.mean())/x.std())

In [58]:
df_total

,,Size,Valuation,Earning,Growth,Momentum,Reverse,Volatility,Liquidity,Beta,pct,tmr_pct
tradedate,stockcode,,,,,,,,,,,
20160901,000006,0.006997,0.089473,-0.045745,-1.479363,-0.087728,-1.252683,-0.766273,0.866205,0.287768,-3.7647,-0.9780
20160902,000006,-0.005833,0.109371,-0.043228,-1.478993,0.068723,-0.894333,-0.813998,0.787954,0.286238,-0.9780,1.8519
20160905,000006,0.021431,0.069815,-0.046086,-1.487748,-0.102191,-0.067457,-0.770585,0.775806,0.231126,1.8519,-0.2424
20160906,000006,-0.017579,0.099410,-0.043202,-1.486913,-0.209023,-0.775281,-0.781667,0.732057,0.206120,-0.2424,0.0000
20160907,000006,-0.017953,0.101448,-0.043574,-1.485085,-0.160938,-0.659035,-0.798811,0.697843,0.208528,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...
20180823,603899,1.915275,-2.243064,1.974181,0.145224,1.759256,1.016371,1.087348,-0.235719,1.734518,-0.7040,3.8825
20180824,603899,1.986504,-2.302853,1.940386,0.133212,1.835860,2.240167,1.215866,-0.180908,1.781104,3.8825,0.9425
20180827,603899,1.953772,-2.285396,1.912034,0.168021,1.736592,1.749358,1.139940,-0.192146,1.588374,0.9425,1.0625


So far, you have security-level returns and exposures. You use today's exposure and tmr's return to estimate unadjusted today's factor portfolio levels/returns. If you are using your in-house risk model to get the factor portfolio levels/returns, you could skip the cross-sectional estimation step, then jump into the algo part. At the end of the day, you still need the security-level exposure to estimate your security-level cov using your risk factor cov, as well as using security-level return to do backtesting.

In [59]:
#Stage 1
def getClusterVar( cov,cItems ):
    #compute variance per cluster
    cov_ = cov.loc[ cItems,cItems ]  # matrix slice
    w_ = getIVP(cov_).reshape(-1,1)  #设定reshape后列数为1，行数-1代表自动匹配计算，w_为 X by 1 的风险分配权重
    cVar = np.dot( np.dot(w_.T, cov_),w_ )[0,0]   # 1byX * XbyX * Xby1
    return cVar

In [60]:
#Stage 2
def getQuasiDiag( link ):
    #执行Quasi-Diagonization的步骤，将因子暴露意义下相似的股票放到一起
    #sort clustered items by distance
    link = link.astype(int) #returns a copy of the array converted to the specified type
    #sortIx记录每一层的组分
    sortIx = pd.Series( [link[-1,0] , link[-1,1]] )
    numItems = link[-1,3]    # number of original items,最后一行为root
    while sortIx.max() >= numItems:
        sortIx.index = range(0 , sortIx.shape[0]*2 , 2)  #make space, step_length = 2
        df0 = sortIx[ sortIx>=numItems ]                 #find clusters，把代表cluster而非originalelement的取出来
        i = df0.index; j = df0.values - numItems
        sortIx[i] = link[j,0]     #item 1
        df0 = pd.Series( link[j,1] , index=i+1)  #循环变量在这里结束时+1
        sortIx = sortIx.append(df0)  #item 2
        sortIx = sortIx.sort_index()  #re-sort
        sortIx.index = range( sortIx.shape[0] ) #re-index
    return sortIx.tolist()

In [61]:
#Stage 3
def getRecBipart( cov, sortIx ):
    # Bottom-up: Define the variance of a continuous subset as the variance of an inverse-variance allocation
    # Top-down: Split allocations btw adjacent subsets in inverse proportion to their aggregated variances
    w = pd.Series(1 , index=sortIx)
    cItems = [sortIx]     #initialize all items in one cluster
    while len(cItems)>0:
        cItems = [ i[j:k] for i in cItems for j,k in ( (0,len(i)//2),(len(i)//2,len(i)) ) if len(i)>1 ]
        # above: bi-section
        for i in range(0, len(cItems), 2):    #parse in pairs
            cItems0 = cItems[i]   #cluster 1
            cItems1 = cItems[i+1] #cluster 2
            cVar0 = getClusterVar( cov,cItems0)
            cVar1 = getClusterVar( cov,cItems1)
            alpha = 1 - cVar0/(cVar0+cVar1)
            w[cItems0] *= alpha    #weight1
            w[cItems1] *= 1-alpha  #weight2
        
    return w   #返回最终标的权重Series

In [62]:
#Stage 0
def correlDist(corr):
    # Output a distance matrix based on correlation, where 0<= d[i,j] <=1
    #合理的距离测度，针对于由相关性定义的原始距离矩阵
    dist = ( (1-corr)/2. )**.5
    return dist

def CalEucliDist(vec1,vec2):  
    #Output euclidean distance
    #vec1,vec2分别为两个np.array
    eucli = np.sqrt( np.sum( np.square(vec1 - vec2) ) )  
    return eucli 

def correlDist_ofDist1(dist):
    # A distance matrix based on original dist matrix
    # Compute the Euclidean distance btw any two col-vectors of dist
    # Therefore, new mat is a distance defined over the ENTIRE metric space,\
    # Rather than a PARTICULAR cross-correlation pair!
    dist_ofDist = np.empty( shape(dist) )
    
    for i in range( 0,shape(dist)(1) ):
        for j in range( i+1,shape(dist)(1) ):
            dist_ofDist[i,j] = CalEucliDist( dist[:,i],dist[:,j] )
            dist_ofDist[j,i] = dist_ofDist[i,j]
            
    dist_ofDist = ssd.squareform(dist_ofDist)
    return dist_ofDist  # 转换为CondensedForm的

#Otherwise, 可以调用scipy包中高维空间距离矩阵的算法
def correlDist_ofDist2(dist):
    dist_ofDist = ssd.pdist( dist, 'euclidean' )  #返回CondensedForm，即只有上三角部分存成数组形式
    
    return dist_ofDist  # CondensedForm的

In [63]:
def getHRP(cov, corr):   #传入两个DataFrame
   
    #Construct a hierarchical portfolio
    #corr,cov = pd.DataFrame(corr),pd.DataFrame(cov)
    dist = correlDist(corr)
    
    #new_dist = ssd.squareform(dist)
    new_dist = correlDist_ofDist2(dist)
    
    link = sch.linkage(new_dist,'single')
    sortIx = getQuasiDiag(link)
    sortIx = corr.index[sortIx].tolist()
    hrp = getRecBipart(cov, sortIx)
    
    return hrp.sort_index()

def getIVP( cov , **kargs):
    # compute the inverse-variance portfolio
    # 10/24解决了 zerodivision 错误
    ivp = 1 / filter_extreme_perc(np.diag(cov) , 2) #ivp为一维数组
    ivp /= ivp.sum()
    return ivp               #最终方差大的部分ivp较小

def getIVP2( cov , **kargs):    #回测IVP策略调用该函数
    # compute the inverse-variance portfolio
    # 10/27解决了 mis-matching index 错误
    ivp = 1 / filter_extreme_perc(np.diag(cov) , 2) #ivp为一维数组
    ivp /= ivp.sum()
    ivp = pd.Series(ivp , index = cov.index)
    return ivp               #最终方差大的部分ivp较小

In [71]:
def getCovCorr(df_win):
    
    # we may have risk model factor levels/returns (even corr) when replicating in 2022 in SF
    #########################################################################################
    
    #win为完整的dataframe，双重索引下的因子暴露和pct
    fac_rets = pd.DataFrame()  #fac_rets最后定位每行为tradedate 每列为stylefactor
    # Fit and summarize WLS regression per day to get 纯因子收益率

    for name, group in df_win.groupby('tradedate'):  #迭代过程中：每个name为tradedate，每个group为一个所有stock的dataframe
        Y = group['tmr_pct']   #次日涨跌幅
        X = group[nfacs]
        X = sm.add_constant(X)
        wls_model = sm.WLS( Y,X, weights = np.sqrt( np.exp(group['Size']) ) )
        results = wls_model.fit()
        fac_rets[name] = results.params #当天的纯因子收益率为一个series，存入dataframe
    #print(name) name记录最后一天的日期
    fac_rets = fac_rets.T   #转置为列名为因子名
    
    fac_rets.drop(['const'],axis=1,inplace=True)
    df_faccov = fac_rets.cov()
    mat_faccov = df_faccov.values
    
    # we may have risk model factor levels/returns (even corr) when replicating in 2022 in SF
    #########################################################################################
    
    
    exposure = df_win[nfacs].xs(name).copy()  #win中最后一天的因子暴露
    exposure = exposure.reset_index()   #把stockcode恢复为正常列
    exposure = exposure.values
    
    
    # get security-level corr
    stock_corr = np.empty( ( len(exposure),len(exposure) ) )# 初始化标的相关性矩阵
    for i in range(len(exposure)):  #从0开始
        stock_corr[i,i] = 1.0
        Std1 = np.dot(exposure[i,1:].dot( mat_faccov), np.transpose(exposure[i,1:]) )**0.5
        for j in range(i+1,len(exposure)):     #从i+1开始
            Std2 = np.dot(exposure[j,1:].dot( mat_faccov), np.transpose(exposure[j,1:]) )**0.5
            stock_corr[i,j] = np.dot( exposure[i,1:].dot( mat_faccov), np.transpose(exposure[j,1:]) )/ (Std1*Std2)
            stock_corr[j,i] = stock_corr[i,j]
    df_stock_corr = pd.DataFrame(stock_corr)
    df_stock_corr['stockcode'] = exposure[:,0]
    df_stock_corr = df_stock_corr.set_index('stockcode')
    df_stock_corr = df_stock_corr.T
    df_stock_corr['stockcode'] = exposure[:,0]
    df_stock_corr = df_stock_corr.set_index('stockcode')
    
    stock_cov = np.dot( exposure[:,1:].dot(mat_faccov) ,exposure[:,1:].T  )
    df_stock_cov = pd.DataFrame(stock_cov)
    df_stock_cov['stockcode'] = exposure[:,0]
    df_stock_cov = df_stock_cov.set_index('stockcode')
    df_stock_cov = df_stock_cov.T
    df_stock_cov['stockcode'] = exposure[:,0]
    df_stock_cov = df_stock_cov.set_index('stockcode')
    
    return df_stock_cov,df_stock_corr

In [72]:
def tail_ratio(returns):
    """Determines the ratiro btw the right95% and left5% tail
     Input:
     returns: pd.Series
          daily/weekly/monthly returns of strategy, noncumulative
     Output:
     tail ratio describing how good or bad the relative tails will lead to
     """
    return np.abs(np.percentile(returns,95)) / np.abs(np.percentile(returns,5))

In [73]:
def computeVaR( returns, T = 10):
    #计算VaR: alpha = 0.01
    VaR_Tday = returns.mean() * T + (-2.33) * returns.std() * T**0.5
    return VaR_Tday

In [74]:
def riskMeasure( returns):
    AnnualRet = returns.mean()*252
    AnnualVol = returns.std() * 252**0.5
    AnnualSharpe = (AnnualRet - 0.0284) / AnnualVol
    return AnnualRet, AnnualVol, AnnualSharpe

In [75]:
def allocAssist( w_yester,r_today):
    #回测辅助函数
    #第一种情况的针对方法：次日收益率字典缺失部分补为默认值1，且多余部分保持不变
    #第二种情况的针对方法：次日收益率字典多余部分截取
    r_base = pd.Series([1]*len(w_yester) , index=w_yester.index)
    r_fix= r_base.mul( r_today,fill_value=1)
    r_fixagain = r_fix.loc[w_yester.index]
    
    return r_fixagain

In [76]:
def go_Backtest( sLength=24, rebal=6):    
    """
    输入 或 视该dataframe作用域为全局
    df_total
    在每个调仓配置时点，用前sLength个交易日数据得出此时最佳权重配置，维持到后rebal个交易日为止
    """
    stats = {'HRP_cul': pd.Series(),'IVP_cul': pd.Series(),'EQ_cul':pd.Series(),\
             'HRP_daily':pd.Series(),'IVP_daily':pd.Series(),'EQ_daily':pd.Series()}
         #字典（key为方法，value为series）后来转为dataframe
    r = pd.DataFrame(columns = ['HRP','IVP','EQ'],index=range(sLength,len(df_total.index.levels[0])) )
    
    pointers = range(sLength,len(df_total.index.levels[0])-rebal+1, rebal)#range范围设定确保最后一周回测数据不会offset
    for pointer in pointers:   #pointer为新周首日
        print('********************************ThisWeekStartFrom:')
        print(pointer)
        # Compute parameter within sample
        lastmonth = df_total.loc[ df_total.index.levels[0][pointer - sLength:pointer] ] #此处要按外层索引tradedate切片
        cov_, corr_ = getCovCorr(lastmonth)     #传入上个月的df_total，返回标的协方差和相关性，行索引为stockcode
        
        # Compute performance out-of-sample
        nextwk = df_total.loc[ df_total.index.levels[0][pointer : pointer + rebal] ] #此处要按外层索引tradedate切片，pointer+rebal为次周首日
        #输入的为样本内参数，得到该周初始各标的权重
        
        w1 = getHRP(cov=cov_, corr=corr_)  
        w2 = getIVP2(cov=cov_, corr=corr_)
        w3 = pd.Series([1/len(w2)]*len(w2) , index=w2.index)  #等权
        #print('============HRP Weight=============')
        #print(w1)
        #print('============IVP Weight=============')
        #print(w2)
        #size2 = size0+size1
        #w3 = pd.Series([1/size2,1/size2,1/size2,1/size2,1/size2,1/size2,1/size2,1/size2,1/size2,1/size2])
        
        #标的 今日/昨日 为  (1 by N ) 的（1+ pct/100）
        #w_为 (N by 1)，周内的每天都要更新为当天净值比重    
        for today in range(pointer, pointer + rebal):  #周内每天循环
            #各标的 今日净值/昨日净值
            r_today = 1 + nextwk['pct'].loc[df_total.index.levels[0][today]]/100
            #解决日度stock_universe变化导致的不一致，输入任一种w即可
            r_fix = allocAssist( w1,r_today)   
            #组合 今日净值/昨日净值
            r.loc[today , 'HRP'] = np.dot(r_fix , w1)
            r.loc[today , 'IVP'] = np.dot(r_fix , w2)
            r.loc[today , 'EQ'] = np.dot(r_fix , w3)
            # .loc['a':'b']会把前后端点行都取到，和list不同
            temp1 = w1 * r_fix
            w1 = temp1 / temp1.sum()  #每天为明天更新w1
            temp2 = w2 * r_fix
            w2 = temp2 / temp2.sum()  #每天为明天更新w2
            temp3 = w3 * r_fix
            w3 = temp3 / temp3.sum()  #每天为明天更新w3
       
        
    #不同方案最终受益 样本外配置方差 和 weekly_TailRatio均值
    r_ = r['HRP'].reset_index(drop=True)
    stats['HRP_cul'] = r_.cumprod()   #累乘出 “累积”组合净值
    stats['HRP_daily'] = r_                 #日度回报时间序列
    HRP_TR = tail_ratio(r_-1)
    HRP_VaR = computeVaR(r_-1)
    HRP_perfo = riskMeasure(r_-1)
    
    r_ = r['IVP'].reset_index(drop=True)
    stats['IVP_cul'] = r_.cumprod()
    stats['IVP_daily'] = r_
    IVP_TR = tail_ratio(r_-1)
    IVP_VaR = computeVaR(r_-1)
    IVP_perfo = riskMeasure(r_-1)
    
    r_ = r['EQ'].reset_index(drop=True)
    stats['EQ_cul'] = r_.cumprod()
    stats['EQ_daily'] = r_
    EQ_TR = tail_ratio(r_-1)
    EQ_VaR = computeVaR(r_-1)
    EQ_perfo = riskMeasure(r_-1)
    #报告结果
    stats = pd.DataFrame.from_dict(stats , orient = 'columns')
    stats.to_csv( 'ZZ成份股_Real回测0525_%s调仓%s.csv' % (str(sLength),str(rebal)) )
    #print("=================== Culmulative Portfolio =====================")
    #cul = stats[['HRP_cul','IVP_cul']].iloc[-1]
    #print(cul)
    print("===================== TailRatio =====================")
    print(HRP_TR,IVP_TR,EQ_TR)    
    print("====================== 99% VaR ======================")
    print(HRP_VaR,IVP_VaR,EQ_VaR) 
    print("== Performance: AnnualRet AnnualVol AnnualSharpe=====")
    print(HRP_perfo,IVP_perfo,EQ_perfo)
    return

In [77]:
go_Backtest( sLength=24, rebal=6)

E:\Richard-Production\AnacondaSoftware\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
E:\Richard-Production\AnacondaSoftware\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


********************************ThisWeekStartFrom:
24
********************************ThisWeekStartFrom:
30
********************************ThisWeekStartFrom:
36
********************************ThisWeekStartFrom:
42
********************************ThisWeekStartFrom:
48
********************************ThisWeekStartFrom:
54
********************************ThisWeekStartFrom:
60
********************************ThisWeekStartFrom:
66
********************************ThisWeekStartFrom:
72
********************************ThisWeekStartFrom:
78
********************************ThisWeekStartFrom:
84
********************************ThisWeekStartFrom:
90
********************************ThisWeekStartFrom:
96
********************************ThisWeekStartFrom:
102
********************************ThisWeekStartFrom:
108
********************************ThisWeekStartFrom:
114
********************************ThisWeekStartFrom:
120
********************************ThisWeekStartFrom:
126
***********************

STOP on May 25 2022

In [43]:
#go_Backtest( sLength=24, rebal=24)   #HS成份股矩阵月度月度调仓

********************************ThisWeekStartFrom:
24
********************************ThisWeekStartFrom:
48
********************************ThisWeekStartFrom:
72
********************************ThisWeekStartFrom:
96
********************************ThisWeekStartFrom:
120
********************************ThisWeekStartFrom:
144
********************************ThisWeekStartFrom:
168
********************************ThisWeekStartFrom:
192
********************************ThisWeekStartFrom:
216
********************************ThisWeekStartFrom:
240
********************************ThisWeekStartFrom:
264
********************************ThisWeekStartFrom:
288
********************************ThisWeekStartFrom:
312
********************************ThisWeekStartFrom:
336
********************************ThisWeekStartFrom:
360
********************************ThisWeekStartFrom:
384
********************************ThisWeekStartFrom:
408
********************************ThisWeekStartFrom:
432
**************

In [26]:
#go_Backtest( sLength=24, rebal=24)   #ZZ成份股矩阵月度月度调仓

********************************ThisWeekStartFrom:
24
********************************ThisWeekStartFrom:
48
********************************ThisWeekStartFrom:
72
********************************ThisWeekStartFrom:
96
********************************ThisWeekStartFrom:
120
********************************ThisWeekStartFrom:
144
********************************ThisWeekStartFrom:
168
********************************ThisWeekStartFrom:
192
********************************ThisWeekStartFrom:
216
********************************ThisWeekStartFrom:
240
********************************ThisWeekStartFrom:
264
********************************ThisWeekStartFrom:
288
********************************ThisWeekStartFrom:
312
********************************ThisWeekStartFrom:
336
********************************ThisWeekStartFrom:
360
********************************ThisWeekStartFrom:
384
********************************ThisWeekStartFrom:
408
********************************ThisWeekStartFrom:
432
**************

In [28]:
#go_Backtest( sLength=24, rebal=6)   #ZZ成份股矩阵月度周度调仓

********************************ThisWeekStartFrom:
24
********************************ThisWeekStartFrom:
30
********************************ThisWeekStartFrom:
36
********************************ThisWeekStartFrom:
42
********************************ThisWeekStartFrom:
48
********************************ThisWeekStartFrom:
54
********************************ThisWeekStartFrom:
60
********************************ThisWeekStartFrom:
66
********************************ThisWeekStartFrom:
72
********************************ThisWeekStartFrom:
78
********************************ThisWeekStartFrom:
84
********************************ThisWeekStartFrom:
90
********************************ThisWeekStartFrom:
96
********************************ThisWeekStartFrom:
102
********************************ThisWeekStartFrom:
108
********************************ThisWeekStartFrom:
114
********************************ThisWeekStartFrom:
120
********************************ThisWeekStartFrom:
126
***********************

In [27]:
#go_Backtest( sLength=24, rebal=6)   #HS成份股矩阵月度周度调仓

********************************ThisWeekStartFrom:
24
********************************ThisWeekStartFrom:
30
********************************ThisWeekStartFrom:
36
********************************ThisWeekStartFrom:
42
********************************ThisWeekStartFrom:
48
********************************ThisWeekStartFrom:
54
********************************ThisWeekStartFrom:
60
********************************ThisWeekStartFrom:
66
********************************ThisWeekStartFrom:
72
********************************ThisWeekStartFrom:
78
********************************ThisWeekStartFrom:
84
********************************ThisWeekStartFrom:
90
********************************ThisWeekStartFrom:
96
********************************ThisWeekStartFrom:
102
********************************ThisWeekStartFrom:
108
********************************ThisWeekStartFrom:
114
********************************ThisWeekStartFrom:
120
********************************ThisWeekStartFrom:
126
***********************

In [22]:
#go_Backtest( sLength=18, rebal=24)   #ZZ成份股矩阵半月度月度调仓

********************************ThisWeekStartFrom:
18
********************************ThisWeekStartFrom:
42
********************************ThisWeekStartFrom:
66
********************************ThisWeekStartFrom:
90
********************************ThisWeekStartFrom:
114
********************************ThisWeekStartFrom:
138
********************************ThisWeekStartFrom:
162
********************************ThisWeekStartFrom:
186
********************************ThisWeekStartFrom:
210
********************************ThisWeekStartFrom:
234
********************************ThisWeekStartFrom:
258
********************************ThisWeekStartFrom:
282
********************************ThisWeekStartFrom:
306
********************************ThisWeekStartFrom:
330
********************************ThisWeekStartFrom:
354
********************************ThisWeekStartFrom:
378
********************************ThisWeekStartFrom:
402
********************************ThisWeekStartFrom:
426
**************

In [23]:
#go_Backtest( sLength=18, rebal=6)   #ZZ成份股矩阵半月度周度调仓

********************************ThisWeekStartFrom:
18
********************************ThisWeekStartFrom:
24
********************************ThisWeekStartFrom:
30
********************************ThisWeekStartFrom:
36
********************************ThisWeekStartFrom:
42
********************************ThisWeekStartFrom:
48
********************************ThisWeekStartFrom:
54
********************************ThisWeekStartFrom:
60
********************************ThisWeekStartFrom:
66
********************************ThisWeekStartFrom:
72
********************************ThisWeekStartFrom:
78
********************************ThisWeekStartFrom:
84
********************************ThisWeekStartFrom:
90
********************************ThisWeekStartFrom:
96
********************************ThisWeekStartFrom:
102
********************************ThisWeekStartFrom:
108
********************************ThisWeekStartFrom:
114
********************************ThisWeekStartFrom:
120
************************

In [20]:
#go_Backtest( sLength=21, rebal=24)   #ZZ成份股矩阵多半月度月度调仓

********************************ThisWeekStartFrom:
21
********************************ThisWeekStartFrom:
45
********************************ThisWeekStartFrom:
69
********************************ThisWeekStartFrom:
93
********************************ThisWeekStartFrom:
117
********************************ThisWeekStartFrom:
141
********************************ThisWeekStartFrom:
165
********************************ThisWeekStartFrom:
189
********************************ThisWeekStartFrom:
213
********************************ThisWeekStartFrom:
237
********************************ThisWeekStartFrom:
261
********************************ThisWeekStartFrom:
285
********************************ThisWeekStartFrom:
309
********************************ThisWeekStartFrom:
333
********************************ThisWeekStartFrom:
357
********************************ThisWeekStartFrom:
381
********************************ThisWeekStartFrom:
405
********************************ThisWeekStartFrom:
429
**************

In [21]:
#go_Backtest( sLength=21, rebal=6)   #ZZ成份股矩阵多半月度周度调仓

********************************ThisWeekStartFrom:
21
********************************ThisWeekStartFrom:
27
********************************ThisWeekStartFrom:
33
********************************ThisWeekStartFrom:
39
********************************ThisWeekStartFrom:
45
********************************ThisWeekStartFrom:
51
********************************ThisWeekStartFrom:
57
********************************ThisWeekStartFrom:
63
********************************ThisWeekStartFrom:
69
********************************ThisWeekStartFrom:
75
********************************ThisWeekStartFrom:
81
********************************ThisWeekStartFrom:
87
********************************ThisWeekStartFrom:
93
********************************ThisWeekStartFrom:
99
********************************ThisWeekStartFrom:
105
********************************ThisWeekStartFrom:
111
********************************ThisWeekStartFrom:
117
********************************ThisWeekStartFrom:
123
************************

In [20]:
#go_Backtest( sLength=27, rebal=24)   #ZZ成份股矩阵一个半月度月度调仓

********************************ThisWeekStartFrom:
27
********************************ThisWeekStartFrom:
51
********************************ThisWeekStartFrom:
75
********************************ThisWeekStartFrom:
99
********************************ThisWeekStartFrom:
123
********************************ThisWeekStartFrom:
147
********************************ThisWeekStartFrom:
171
********************************ThisWeekStartFrom:
195
********************************ThisWeekStartFrom:
219
********************************ThisWeekStartFrom:
243
********************************ThisWeekStartFrom:
267
********************************ThisWeekStartFrom:
291
********************************ThisWeekStartFrom:
315
********************************ThisWeekStartFrom:
339
********************************ThisWeekStartFrom:
363
********************************ThisWeekStartFrom:
387
********************************ThisWeekStartFrom:
411
********************************ThisWeekStartFrom:
435
**************

In [21]:
#go_Backtest( sLength=30, rebal=24)   #ZZ成份股矩阵一个半半月度月度调仓

********************************ThisWeekStartFrom:
30
********************************ThisWeekStartFrom:
54
********************************ThisWeekStartFrom:
78
********************************ThisWeekStartFrom:
102
********************************ThisWeekStartFrom:
126
********************************ThisWeekStartFrom:
150
********************************ThisWeekStartFrom:
174
********************************ThisWeekStartFrom:
198
********************************ThisWeekStartFrom:
222
********************************ThisWeekStartFrom:
246
********************************ThisWeekStartFrom:
270
********************************ThisWeekStartFrom:
294
********************************ThisWeekStartFrom:
318
********************************ThisWeekStartFrom:
342
********************************ThisWeekStartFrom:
366
********************************ThisWeekStartFrom:
390
********************************ThisWeekStartFrom:
414
********************************ThisWeekStartFrom:
438
*************

In [22]:
#go_Backtest( sLength=27, rebal=6)   #ZZ成份股矩阵一个半月度周度调仓

********************************ThisWeekStartFrom:
27
********************************ThisWeekStartFrom:
33
********************************ThisWeekStartFrom:
39
********************************ThisWeekStartFrom:
45
********************************ThisWeekStartFrom:
51
********************************ThisWeekStartFrom:
57
********************************ThisWeekStartFrom:
63
********************************ThisWeekStartFrom:
69
********************************ThisWeekStartFrom:
75
********************************ThisWeekStartFrom:
81
********************************ThisWeekStartFrom:
87
********************************ThisWeekStartFrom:
93
********************************ThisWeekStartFrom:
99
********************************ThisWeekStartFrom:
105
********************************ThisWeekStartFrom:
111
********************************ThisWeekStartFrom:
117
********************************ThisWeekStartFrom:
123
********************************ThisWeekStartFrom:
129
***********************

In [23]:
#go_Backtest( sLength=30, rebal=6)   #ZZ成份股矩阵一个半半月度周度调仓

********************************ThisWeekStartFrom:
30
********************************ThisWeekStartFrom:
36
********************************ThisWeekStartFrom:
42
********************************ThisWeekStartFrom:
48
********************************ThisWeekStartFrom:
54
********************************ThisWeekStartFrom:
60
********************************ThisWeekStartFrom:
66
********************************ThisWeekStartFrom:
72
********************************ThisWeekStartFrom:
78
********************************ThisWeekStartFrom:
84
********************************ThisWeekStartFrom:
90
********************************ThisWeekStartFrom:
96
********************************ThisWeekStartFrom:
102
********************************ThisWeekStartFrom:
108
********************************ThisWeekStartFrom:
114
********************************ThisWeekStartFrom:
120
********************************ThisWeekStartFrom:
126
********************************ThisWeekStartFrom:
132
**********************

In [20]:
#go_Backtest( sLength=33, rebal=24)   #ZZ成份股矩阵一个半半半月度月度调仓

********************************ThisWeekStartFrom:
33
********************************ThisWeekStartFrom:
57
********************************ThisWeekStartFrom:
81
********************************ThisWeekStartFrom:
105
********************************ThisWeekStartFrom:
129
********************************ThisWeekStartFrom:
153
********************************ThisWeekStartFrom:
177
********************************ThisWeekStartFrom:
201
********************************ThisWeekStartFrom:
225
********************************ThisWeekStartFrom:
249
********************************ThisWeekStartFrom:
273
********************************ThisWeekStartFrom:
297
********************************ThisWeekStartFrom:
321
********************************ThisWeekStartFrom:
345
********************************ThisWeekStartFrom:
369
********************************ThisWeekStartFrom:
393
********************************ThisWeekStartFrom:
417
********************************ThisWeekStartFrom:
441
=============

In [21]:
#go_Backtest( sLength=33, rebal=6)   #ZZ成份股矩阵一个半半半月度周度调仓

********************************ThisWeekStartFrom:
33
********************************ThisWeekStartFrom:
39
********************************ThisWeekStartFrom:
45
********************************ThisWeekStartFrom:
51
********************************ThisWeekStartFrom:
57
********************************ThisWeekStartFrom:
63
********************************ThisWeekStartFrom:
69
********************************ThisWeekStartFrom:
75
********************************ThisWeekStartFrom:
81
********************************ThisWeekStartFrom:
87
********************************ThisWeekStartFrom:
93
********************************ThisWeekStartFrom:
99
********************************ThisWeekStartFrom:
105
********************************ThisWeekStartFrom:
111
********************************ThisWeekStartFrom:
117
********************************ThisWeekStartFrom:
123
********************************ThisWeekStartFrom:
129
********************************ThisWeekStartFrom:
135
**********************